In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_data(task):
    if task == "3.1":
        x1 = [-1, -1, 1, -1, 1, -1, -1, 1]  # [-1., -1., 1., -1., 1., -1., -1., 1.]
        x2 = [-1, -1, -1, -1, -1, 1, -1, -1]  # [-1., -1., -1., -1., -1., 1., -1., -1.]
        x3 = [-1, 1, 1, -1, -1, 1, -1, 1]  # [-1., 1., 1., -1., -1., 1., -1., 1.]
        memory_patterns = [x1, x2, x3]
        x1d = [1, -1, 1, -1, 1, -1, -1, 1] # [1., -1., 1., -1., 1., -1., -1., 1.]
        x2d = [1, 1, -1, -1, -1, 1, -1, -1] # [1., 1., -1., -1., -1., 1., -1., -1.]
        x3d = [1, 1, 1, -1, 1, 1, -1, 1]#[1., 1., 1., -1., 1., 1., -1., 1.]
        test_patterns = [x1d, x2d, x3d]
        x1dd = [1, 1, -1, 1, 1, -1, -1, -1]
        x2dd = [1, 1, -1, 1, 1, 1, 1, -1]
        x3dd = [1, -1, 1, 1, 1, -1, -1, 1]
        dissimilar_patterns = [x1dd, x2dd, x3dd]
        return np.array(memory_patterns), np.array(test_patterns), np.array(dissimilar_patterns)


    data = np.fromfile("pict.dat",sep=",")
    pict = np.zeros((11, 1024))
    #img = np.zeros((11, 32, 32))
    for k in range(11):
        pict[k, :] = data[k*1024:(k+1)*1024]
        #img[k,:] = pict[k].reshape(32,32)

    if task == "3.2":
        #visualize_img(pict[0:3], ["p1", "p2", "p3"])
        return pict[0:3], pict[9:11]

    if task == "3.4":
        # add noise
        gaussian_noise = np.random.normal(0, 2, 1024)#(1024,)
        pict_with_noise = []# 3 * 7 * 1024
        noise_percent = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
        for percent in noise_percent:
            with_noise = []
            for i in range(3):
                noise = np.zeros(1024)
                noise[0: int(1024*percent)] = gaussian_noise[0: int(1024*percent)]
                np.random.shuffle(noise)
                with_noise.append(noise+pict[i])
            #visualize_img(np.array(with_noise),["p1 with {}% noise".format(percent*100), "p2 with {}% noise".format(percent*100), "p3 with {}% noise".format(percent*100)])# visualization
            pict_with_noise.append(np.array(with_noise))
        return pict[0:3], np.array(pict_with_noise)

    if task == "3.5":
        return pict[0:7],

def visualize_img(pict, info=None):
    plt.figure()
    num = pict.shape[0]
    if num==1:
        img = pict[0].reshape(32, 32)
        plt.imshow(img)
        plt.title(info[0])
    else:
        f, ax = plt.subplots(1, num)
        for i in range(num):
            img = pict[i].reshape(32, 32)
            ax[i].set_title("{}".format(info[i]))
            ax[i].imshow(img)
    plt.show()


class Hopfield_network():
    def __init__(self, dim):
        self.W = None
        self.dim = dim
        self.max_iter = 1000

    def weight_update(self, X):
        self.W = np.zeros((self.dim, self.dim))
        for i in range(X.shape[0]):
            self.W += np.dot(X[i].reshape(-1, 1), X[i].reshape(-1, 1).T)
        #self.W /= self.dim

    def weight_symmetric(self):
        self.weight_normal()
        self.W = 0.5 * (self.W + self.W.T)

    def weight_normal(self):
        self.W = np.random.normal(0, 1, (self.dim, self.dim))

    def energy(self, init_state):
        wst = np.dot(self.W, init_state)
        return -np.dot(init_state.T, wst)[0][0]

    def sync_update(self, init_state):
        old_state = np.copy(init_state)#(1024,1)
        new_state = np.zeros((init_state.shape[0], old_state.shape[1]))
        it = 0
        while(it < self.max_iter):
            #wx =  #(1024, 1)
            new_state = self.sign(np.dot(self.W, old_state))#.reshape(1, -1)) #(1024,1)
            #print(new_state)
            if np.equal(new_state, old_state).all() == True:
                old_state = np.copy(new_state)
                print("Synchronous: Converge in {} iterations".format(it))
                break
            old_state = np.copy(new_state)
            it += 1
        #print("Cannot converge in {} iterations".format(self.max_iter))
        return old_state.reshape(1, -1)[0]

    def asyn_update(self, init_state, show_energy=False):
        old_state = np.copy(init_state)
        new_state = np.zeros((init_state.shape[0], old_state.shape[1]))
        self._trace = [[], []]
        for i in range(self.max_iter):
            if show_energy == True:
                print("iteration", i, " --- ", self.energy(old_state))
            index = np.random.randint(0, self.dim, self.dim)
            for j in range(self.dim):
                new_state[index[j]] = self.sign(np.dot(self.W[index[j]], old_state))
            if np.array_equal(new_state, old_state):
                old_state = np.copy(new_state)
                print("Asynchronous: Converge in {} iterations".format(i))
                return old_state.reshape(1, -1)[0]
            old_state = np.copy(new_state)#(1024,1)
            #if i % 100 == 0:
                #visualize_img(np.array([old_state.reshape(1, -1)[0]]), ["{} iterations".format(i)])
        print("Cannot converge in {} iterations".format(self.max_iter))
        #visualize_img(np.array([old_state.reshape(1, -1)[0]]), ["converge at {} iterations".format(i)])
        return old_state.reshape(1, -1)[0]

    def sign(self, wx):
        #return np.sign(wx)
        return np.where(wx<=0, -1, 1)

    def check_attractions(self, init_state):
        state = self.sign(np.dot(self.W, init_state).reshape(1, -1))
        return np.equal(state, init_state).all()



"""
Main
"""
"""3.1"""

train, test, test_2 = get_data("3.1")
net = Hopfield_network(train.shape[1])
net.weight_update(train)
for i in range(test.shape[0]):
    output = net.sync_update(test[i].reshape(-1, 1))#(8, 1)
    print(" Input: {}; \n Output: {}; \n Stored: {}.\n".format(test[i], output, train[i]))
# find all attractions
count = 0
for j in range(256):#2^8
    ele = np.array([(j >> y & 1) for y in range(8 - 1, -1, -1)])
    state = np.where(ele==1, 1, -1)
    if net.check_attractions(state):#(8,)
        count += 1
        print(state)
print("#attractions = ", count)
# test more dissimilar starting pattern
for i in range(test_2.shape[0]):
    output = net.sync_update(test_2[i].reshape(-1, 1))# (8, 1)
    print(" Input: {};  Output: {}".format(test_2[i], output))


"""3.2"""
'''
train, test = get_data("3.2")
net = Hopfield_network(train.shape[1])
net.weight_update(train)
res1 = []
for i in range(test.shape[0]):
    output = net.sync_update(test[i].reshape(-1, 1))#(1024,1)
    res1.append(output)
visualize_img(np.array([test[0],res1[0], test[1],res1[1]]), ["p10", "updated p10", "p11", "updated p11"])
# select units randomly
res2 = []
for i in range(test.shape[0]):
    output = net.asyn_update(test[i].reshape(-1, 1))#(1024,1)
    res2.append(output)
visualize_img(np.array([test[0],res2[0], test[1],res2[1]]), ["p10", "after", "p11", "after"])
'''

"""3.3"""
'''
# energy at the different attractions
train, test = get_data("3.2")
net = Hopfield_network(train.shape[1])
net.weight_update(train)
for i in range(train.shape[0]):
    print("energy at p{}: ".format(i+1), net.energy(train[i].reshape(-1, 1)))
# energy at the points of the distorted patterns
for i in range(test.shape[0]):
    print("energy at p{}: ".format(i+10), net.energy(test[i].reshape(-1, 1)))
# how energy changes from iteration to iteration in sequential update rule
for i in range(test.shape[0]):
    print("energy change in p{}: ".format(i+10))
    output = net.asyn_update(test[i].reshape(-1, 1), show_energy=True)
# normally distributed random weight
net.weight_normal()
for i in range(test.shape[0]):
    print("energy change in p{} with randomly normal weight matrix".format(i+10))
    output = net.asyn_update(test[i].reshape(-1, 1))
# symmetric weight matrix
net.weight_symmetric()
for i in range(test.shape[0]):
    print("energy change in p{} with symmetric weight matrix".format(i+10))
    output = net.asyn_update(test[i].reshape(-1, 1))
'''

"""3.4"""
'''
# how much noise can be removed
train, test = get_data("3.4")
net = Hopfield_network(train.shape[1])
net.weight_update(train)
acc = np.zeros((10, 3))
for m in range(50):
    train, test = get_data("3.4")
    for i in range(test.shape[0]): # 10 noise
        print("Adding {}% noise to the patterns".format((i+1)*10))
        res1 = []
        for j in range(test.shape[1]): # 3 patterns
            output = net.sync_update(test[i][j].reshape(-1, 1))
            if np.equal(output, train[j]).all() == True:
                acc[i, j] += 1
        #print("recover rate of p{0} with {1}% noise: {2}".format(j + 1, (i + 1)*10, acc/50))
print(acc/50)
            #res1.append(output)
        #visualize_img(np.array([test[i][0], res1[0], test[i][1], res1[1], test[i][2], res1[2]]), ["{}% noise".\
                      #format((i+1)*10),"result", "{}% noise".format((i+1)*10),"result", "{}% noise".format((i+1)*10),"result"])
'''
"""3.5"""
# how many patterns could be safely stored


In [ ]:

import numpy as np
import matplotlib
#matplotlib.use("TkAgg")
from matplotlib import pyplot as plt
import itertools

class Hopfield():
    """
    implementation of a Hopfield network
    """

    def fit(self,X,diag=True,bias=False):
        self.X = X # patterns
        self.N = X.shape[0]
        self.D = X.shape[1] # number of neurons 
        self.diag = diag
        self.bias = bias
        self.maxIters = np.log(self.N)

        self.learning()

    def learning(self):
        self.W = np.dot(self.X.T, self.X) / 1#/ self.N
        if self.diag == False:
            np.fill_diagonal(self.W, 0)

    def recalls(self,Xmatrix,sync=True):
        self.recalled = []
        for idx, Xvector in enumerate(Xmatrix):
            self.recalled.append(self.recall(Xvector=Xvector,Xindex=idx,sync=sync))

    def recall(self,Xvector,Xindex=0,sync=True):
        activations_prev = Xvector # t-1
        activations_curr = 0 # t
        self.sync = sync
        
        for iter in range(int(self.maxIters)+1):
            activations_curr = self.update_neurons(activations_prev)
            if np.all(activations_curr == activations_prev):
                '''
                try +1 iteration
                '''
                activations_curr = self.update_neurons(activations_prev)
                # print("Pattern",Xindex,"converged in",iter+1,"iterations!") # convergence
                break
            activations_prev = activations_curr
        # print(sum(self.X[Xindex,:] == self.activations_prev)/self.D) # accuracy of obs
        return activations_curr

    def update_neurons(self,activations):
        if self.sync == True: # synchronous / batch update
            new = np.where(np.dot(self.W, activations.T).T > 0, 1,-1)
        
        else: # asynchronous / sequential update
            new = activations.copy() # need copy due to argument
            order = np.arange(self.D) #nr of nodes
            # np.random.shuffle(order)
            for i in order:
                new[i] = np.where(np.dot(self.W[i,:], new.T) > 0, 1,-1)

        return new

    def getAttractors(self, Xmatrix,sync=True):
        count = 0
        # print(sync)
        for p in Xmatrix:
            potential_attractor = self.recall(Xvector=p,sync=sync)
            if np.all(potential_attractor == p.T):
                print(potential_attractor)
                count += 1
        print("#attractions = ", count)

def visualize_img(pict, info=None):
    plt.figure()
    num = pict.shape[0]
    if num==1:
        img = pict[0].reshape(32, 32)
        plt.imshow(img)
        plt.title(info[0])
    else:
        f, ax = plt.subplots(1, num)
        for i in range(num):
            img = pict[i].reshape(32, 32)
            ax[i].set_title("{}".format(info[i]))
            ax[i].imshow(img)
    plt.show()

def distortData(data,pNoise):
    noisyData = np.copy(data)
    for i in range(data.shape[0]):
        idx = np.arange(data.shape[1])
        np.random.shuffle(idx)
        idxNoise = idx[:int(data.shape[1]*pNoise)]
        for j in idxNoise:
            noisyData[i,j] = np.where(noisyData[i,j] == 1,-1,1)
    return noisyData

def capacity(data="random",diag=True,sync=True,out="list",noise=False,bias=False):
    if data != "random":
        for prob in [0.0, 0.05,0.10,0.15,0.20,0.25,0.30,0.40,0.50]:
            noisyData = distortData(data,pNoise=prob)
            # visualize_img(np.array([data[0],noisyData[0],data[1],noisyData[1],data[2],noisyData[2],data[3],noisyData[3]]), ["p1", "p1Noisy","p2", "p2Noisy","p3", "p3Noisy","p4", "p4Noisy",])
            accs = []
            for i in range(data.shape[0]):
                h = Hopfield()
                trainData = data[:i+1:]
                h.fit(X=trainData,diag=diag) #(i+1,1024)
                noiseData = noisyData[:i+1:]
                h.recalls(Xmatrix=noiseData,sync=sync)
                
                accuracy = 0
                for t, r in zip(trainData, h.recalled):
                    if np.all(t == r):
                        accuracy += 1
                accs.append(accuracy / noiseData.shape[0])
            
            if out == "list": 
                print("pNoise: ",prob,accs)
            elif out == "#stored": # print number of stored patterns
                print("pNoise: {} Stored patterns: {}".format(prob,np.count_nonzero(np.array(accs) == 1.0)))
    
    else: # random patterns
        # visualize_img(np.array([data[0],noisyData[0],data[1],noisyData[1],data[2],noisyData[2],data[3],noisyData[3]]), ["p1", "p1Noisy","p2", "p2Noisy","p3", "p3Noisy","p4", "p4Noisy",])
        N = 30
        Accs = np.zeros(N)
        n = 100 # in order to get averages
        for i in range(n):
            if bias == False:
                randomData = np.where(np.random.normal(size=(N,100)) > 0, 1, -1)
            else:
                randomData = np.where(0.5+np.random.normal(size=(N,100)) > 0, 1, -1)
            accs = []
            for i in range(N):
                h = Hopfield()
                trainData = randomData[:i+1:]
                h.fit(X=trainData,diag=diag) #(i+1,1024)
                if noise == False:
                    #h.recalls(Xmatrix=trainData,sync=sync)
                    h.recalls(Xmatrix=trainData,sync=False)
                else:
                    # print("noise")
                    prob = 0.1
                    noisyData = distortData(randomData,pNoise=prob)
                    h.recalls(Xmatrix=noisyData,sync=sync)
                
                accuracy = 0
                for t, r in zip(trainData, h.recalled):
                    if np.all(t == r):
                        accuracy += 1
                accs.append(accuracy / trainData.shape[0])

            Accs += np.array(accs)
        Accs /= n

        if out == "list": 
            print(Accs)
        elif out == "#stored": # print number of stored patterns
            print("Stored patterns: {}".format(np.count_nonzero(np.array(Accs) == 1.0)))
            print(Accs)
            plt.figure()
            plt.plot(Accs)
            # plt.errorbar(frac_training, means, yerr=stds, fmt='o-') #,label='sigma=0.03')
            # plt.legend(loc='upper right')
            plt.xlabel('Number of patterns')
            plt.ylabel('Accuracy')
            plt.show()

if __name__ == "__main__":
# ---------------------------------------------------------------------------------------------- #
#  2.2. Hopfield network recall
# ---------------------------------------------------------------------------------------------- #
    print("\n2.2. Hopfield network recall")
    x1 = [-1,-1, 1,-1, 1,-1,-1, 1]
    x2 = [-1,-1,-1,-1,-1, 1,-1,-1]
    x3 = [-1, 1, 1,-1,-1, 1,-1, 1]
    X = np.array([x1,x2,x3])

    h = Hopfield()
    h.fit(X=X)
    h.recalls(Xmatrix=X)
# ---------------------------------------------------------------------------------------------- #
#  3.1. Convergence and attractors
# ---------------------------------------------------------------------------------------------- #
    print("\n3.1. Convergence and attractors")
    x1d = [ 1,-1, 1,-1, 1,-1,-1, 1]
    x2d = [ 1, 1,-1,-1,-1, 1,-1,-1]
    x3d = [ 1, 1, 1,-1, 1, 1,-1, 1]
    Xd = np.array([x1d,x2d,x3d])
# Apply the update rule repeatedly until you reach a stable fixed point.
# Did all the patterns converge towards stored patterns?
    h.recalls(Xmatrix=Xd)

# How many attractors are there in this (256 possible combinations) network? Hint: automate the searching.
    print("\nAttractors")
    X256 = np.array(list(itertools.product([-1,1],repeat=8)))
    h = Hopfield()
    h.fit(X=X)
    h.getAttractors(Xmatrix=X256,sync=False) # get 6 attractors with async and zeros on diag

# What happens when you make the starting pattern even more dissimilar
# to the stored ones (e.g. more than half is wrong)?
    print("\nDissimilar patterns:")
    x1d2 = [ 1, 1,-1, 1,-1,-1,-1, 1]
    x2d2 = [ 1, 1, 1, 1, 1, 1,-1,-1]
    x3d2 = [ 1,-1,-1, 1, 1,-1,-1, 1]
    Xd2 = np.array([x1d2,x2d2,x3d2])
    h.recalls(Xmatrix=Xd2)
    # Do not get back the stored patterns

# ---------------------------------------------------------------------------------------------- #
#  3.2. Sequential Update
# ---------------------------------------------------------------------------------------------- #
    print("\nSequential update")
    SYNC = False
    pictData = np.loadtxt('./pict.dat', delimiter=",", dtype=int).reshape(-1, 1024)
    p1p2p3 = pictData[:3:] #(3,1024)
    h = Hopfield()
    h.fit(X=p1p2p3)
    h.recalls(Xmatrix=p1p2p3,sync=SYNC)

    # Check that the three patterns are stable.
    res1 = []
    for i in range(p1p2p3.shape[0]):
        res1.append(h.recall(p1p2p3[i],sync=SYNC))
    # visualize_img(np.array([p1p2p3[0],res1[0], p1p2p3[1],res1[1], p1p2p3[2],res1[2]]), ["p1", "recalled p1", "p2", "recalled p2", "p3", "recalled p3"])

    # Can the network complete a degraded pattern? Try the pattern p10, which
    # is a degraded version of p1, or p11 which is a mixture of p2 and p3.
    p10p11 = pictData[-2:]

    res1 = []
    for i in range(p10p11.shape[0]):
        res1.append(h.recall(p10p11[i],sync=SYNC)) #(1024,1)
    # visualize_img(np.array([p1p2p3[0],p10p11[0],res1[0],p1p2p3[1],p1p2p3[2],p10p11[1],res1[1]]), ["p1", "p10","recalled p10", "p2", "p3", "p11", "recalled p11"])
    # p10 == p1 with batch, not seq. p11 == p2 with seq, otherwise nothing.


# ---------------------------------------------------------------------------------------------- #
#  3.3. Energy
# ---------------------------------------------------------------------------------------------- #

# ---------------------------------------------------------------------------------------------- #
#  3.4. Distortion Resistance
# ---------------------------------------------------------------------------------------------- #

# ---------------------------------------------------------------------------------------------- #
#  3.5. Capacity
# ---------------------------------------------------------------------------------------------- #
    print("Capacity")
# How many patterns could safely be stored? Was the drop in performance gradual or abrupt?

    # capacity(data=pictData) # three
    # capacity(data=pictData,sync=False) # three

# Try to repeat this with learning a few random patterns
# instead of the pictures and see if you can store more.

    # randomData = np.where(np.random.normal(size=(11,1024)) > 0, 1, -1)
    # capacity(data=randomData)
    # capacity(data=randomData,sync=False)

# It has been shown that the capacity of a Hopfield network
# is around 0.138N. How do you explain the difference
# between random patterns and the pictures?

# What happens with the number of stable patterns as more are learned?
    # They decrease and then increase (for batch and sequential)

    capacity(data="random",out="#stored",noise=False)
    # capacity(data="random",out="#stored",noise=False,sync=False)

# What happens if convergence to the pattern from a noisy version (a few fliipped units)
# is used? What does the different behavior for large number of patterns mean?
    
    # They decrease (for batch and sequential)
    # capacity(data="random",out="#stored",noise=True)
    # capacity(data="random",out="#stored",noise=True,sync=False)

# Remove self-connections. What is the maximum number of retrievable patterns for this network?
    
    # capacity(data="random",out="#stored",noise=True,diag=False) # 4 patterns on average / 6 with self connections
    
    # capacity(data="random",out="#stored",noise=True,diag=False,bias=True) # 4 even less patterns




2.2. Hopfield network recall

3.1. Convergence and attractors

Attractors
[-1 -1 -1 -1 -1  1 -1 -1]
[-1 -1 -1 -1  1 -1 -1 -1]
[-1 -1  1 -1 -1 -1 -1  1]
[-1 -1  1 -1 -1  1 -1  1]
[-1 -1  1 -1  1 -1 -1  1]
[-1  1  1 -1 -1  1 -1  1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1  1 -1  1 -1]
[ 1 -1  1  1  1 -1  1  1]
[ 1  1 -1  1 -1 -1  1 -1]
[ 1  1 -1  1 -1  1  1 -1]
[ 1  1 -1  1  1 -1  1 -1]
[ 1  1  1  1 -1  1  1  1]
[ 1  1  1  1  1 -1  1  1]
#attractions =  14

Dissimilar patterns:

Sequential update


OSError: ignored